In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
openasl = pd.read_csv('openasl-v1.0.tsv', sep='\t')
openasl.head()

/tmp/ipykernel_14040/844222082.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  openasl = pd.read_csv('openasl-v1.0.tsv', sep='\t')


,vid,yid,start,end,raw-text,tokenized-text,gloss,split
0,Mci9oyb5V2E-00:00:06.000-00:00:06.589,Mci9oyb5V2E,00:00:06.000,00:00:06.589,Hello!,hello,NaN,train
1,RjJSxzMkC90-00:00:38.905-00:00:39.975,RjJSxzMkC90,00:00:38.905,00:00:39.975,Then it was time.,then it was time,NaN,train
2,RjJSxzMkC90-00:00:37.970-00:00:38.905,RjJSxzMkC90,00:00:37.970,00:00:38.905,"I said, ""oh.""",i said oh,NaN,train
3,RjJSxzMkC90-00:00:25.123-00:00:27.524,RjJSxzMkC90,00:00:25.123,00:00:27.524,Bush through relay.,bush through relay,NaN,train
4,RjJSxzMkC90-00:00:14.083-00:00:17.094,RjJSxzMkC90,00:00:14.083,00:00:17.094,The secretary called me to come.,the secretary called me to come,NaN,train


Original Split between train, dev and test

In [4]:
openasl['split'].value_counts(normalize=True) * 100

split
train    98.025808
test      0.991668
valid     0.982524
Name: proportion, dtype: float64

run facial recognition on a sample video

frame extraction

In [5]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [6]:
import cv2
import numpy as np
from pathlib import Path

In [7]:
videoname = input()
cap = cv2.VideoCapture(videoname)
total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
indices = np.linspace(200, total - 1, 20).astype(int)

In [8]:
saveto = input()
for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
        ok, frame = cap.read()
        if not ok:
            continue

        frame_name = f"{videoname}_{idx}.jpg"
        cv2.imwrite(str(Path(saveto) / frame_name), frame)

Load LVFace

In [9]:
from inference_onnx import LVFaceONNXInferencer

In [15]:
## Initialize the inferencer from inference_onnx.py
inferencer = LVFaceONNXInferencer(
   model_path="LVFace-B_Glint360K.onnx",  # Path to your ONNX model
   use_gpu=False # Set to False for CPU-only 
)

def get_embedding(saveto, videoname):
      emblist = []
      for frame in Path(saveto).glob(videoname + "*.jpg"):
            emb = inferencer.infer_from_image(str(frame))
            emb = emb.squeeze()
            emblist.append((emb))
      return emblist

def get_average(emb):
      emb = np.stack(emb, axis = 0)
      avg = np.mean(emb, axis = 0)
      avg = avg / np.linalg.norm(avg)
      return avg

avg_emb = [get_average(get_embedding("frames1", "videoexample.mp4")), get_average(get_embedding("frames2", "videoexample2.mp4"))]

avg_embs = np.stack(avg_emb, axis=0)


simmatrix = np.dot(avg_embs,avg_embs.T)

simmatrix

array([[0.99999994, 0.24117297],
       [0.24117297, 0.99999976]], dtype=float32)

STEM Wiki

In [ ]:
stem = pd.read_csv('all.csv')
stem.head()

,user,articleName,sectionIndex,sentenceIndex,filename,sentence,fs_span,fs_text
0,62ab763c2653e8005020f166,Acid catalysis,0,0,01655119426829943-2053312_0_0.mp4,"In acid catalysis and base catalysis, a chemic...","[(80, 100), (115, 173), (227, 260), (409, 432)...","['acid', 'catalysis', 'base', 'acid', 'base']"
1,62ab763c2653e8005020f166,Acid catalysis,0,1,5813571323095919-2053312_0_1.mp4,"By Brønsted–Lowry acid–base theory, the acid i...","[(64, 199), (224, 295), (336, 356)]","['Brønsted–Lowry', 'acid–base', 'acid']"
2,62ab763c2653e8005020f166,Acid catalysis,0,2,8261011694304976-2053312_0_2.mp4,Typical reactions catalyzed by proton transfer...,"[(220, 285), (316, 362)]","['esterfications', 'aldol']"
3,62ab763c2653e8005020f166,Acid catalysis,0,3,023397513349495913-2053312_0_3.mp4,"In these reactions, the conjugate acid of the ...","[(86, 100), (105, 186), (198, 215), (259, 336)...","['the', 'conjugate', 'acid', 'carbonyl', 'elec..."
4,62ab763c2653e8005020f166,Acid catalysis,0,4,2738300105048981-2053312_0_4.mp4,Depending on the chemical species that act as ...,"[(131, 144), (169, 189), (204, 218), (243, 317...","['act', 'acid', 'be', 'catalytic', 'catalysis'..."


count signers

In [ ]:
stem['user'].value_counts()

user
62ab763c2653e8005020f166    38
62a0cece9bbab800502f2e90    38
62a25e632e0bec00503747ad    38
62ab8ca14eede20050bef000    36
62a20629ff311c0050d35493    35
6231157f9e3b770048c76dd4    34
629f640e0a2f100050eeaa6a    34
62ac8d364eede20050bef369    33
62a6a8a1cba00d00501964df    33
62a1edef6110330050235169    31
624b3778f1a4ec0048766482    31
62acd9b4b7e7010050c6bb5e    31
62a90031c0bfef0050fdbef7    28
62a27b0dff311c0050d3572e    27
62a0d78a9bbab800502f2eb5    26
Name: count, dtype: int64